Creating yearly and monthly tiffs for the India team to evaluate the feasibility of the modelled data to their envisioned use cases.

In [19]:
import xarray as xr
import rioxarray as rxr
import pandas as pd
import os
import glob
import netCDF4
import geopandas as gpd
import datetime
# import seaborn as sns

### Get data
PM2.5 data generated by the full model are downloaded from zenodo (https://zenodo.org/records/13694585); all files matching full_model_YYYY.zip where downloaded.

In [20]:
data_path = '../../../../on_zenodo/'

# find all zip files
files = glob.glob(data_path + 'full_model_*.zip')

# for each file, check if unzipped folder exists, and it not, unzip
for file in files:
    if os.path.exists(file[:-4]):
        print(f"Already unzipped {file}")
    else:
        print(f"Unzipping {file}")
        os.system(f"unzip {file} -d {file[:-4]}")

# find all nc4 files that match V01FL_PM25_India_YYYYMMDD.nc4
file_list = glob.glob(data_path + 'full_model_*/*/V01FL_PM25_India_*.nc4')

# first file in 2018 is from 10.7., last file in 2023 30.9.
# calculate number of days
days =  (datetime.datetime(2023, 9, 30) - datetime.datetime(2018, 7, 10)).days + 1
# check that number of files matches number of days

assert len(file_list) == days, f"Number of files {len(file_list)} does not match number of days {days}"

Already unzipped ../../../../on_zenodo/full_model_2020.zip
Already unzipped ../../../../on_zenodo/full_model_2023.zip
Already unzipped ../../../../on_zenodo/full_model_2018.zip
Already unzipped ../../../../on_zenodo/full_model_2021.zip
Already unzipped ../../../../on_zenodo/full_model_2022.zip
Already unzipped ../../../../on_zenodo/full_model_2019.zip


### Read data

In [21]:
# Create a function which adds a time dimension to a DataArray, and fill it with a arbitrary date:
# added using the filename to get the datestring
def add_time_dim(xds):
    filename = xds.encoding['source']
    date_str = os.path.basename(filename).split('_')[-1][:8]
    xds = xds.expand_dims(time = [datetime.datetime.strptime(date_str, '%Y%m%d')])
    return xds

# ds = xr.open_dataset(file_list[0], engine='netcdf4', decode_coords="all")
file_list.sort()
# open all files, and add time dimension
ds = xr.open_mfdataset(file_list, combine='nested', concat_dim='time', decode_coords="all", engine='netcdf4', preprocess = add_time_dim)


### Calculate temporal means and save to geotiffs

In [22]:
out_data_path = '../../../../sample_data'
os.makedirs(out_data_path, exist_ok=True)

#### Annual means

In [ ]:
# calculate yearly mean
ds_yearly = ds.resample(time='YS').mean()

# # plot one year for sanity check
# ds_yearly.isel(time=0)['pm25_pred'].plot()

# save to geotiff

for time in ds_yearly.time.values:

    # select pm25 variable and year for file
    data_out = ds_yearly.sel(time=time)['pm25_pred']

    # full filename
    filename_out = os.path.join(out_data_path, 'yearly', f'PM25_{ pd.to_datetime(time).strftime('%Y')}.tif')

    # save the file
    os.makedirs(os.path.dirname(filename_out), exist_ok=True)
    data_out.rio.to_raster(filename_out, driver='GTiff')

#### Yearly means for financial means

In [33]:
# calculate yearly mean, starting from 1st April of each year
ds_yearly = ds.resample(time='AS-APR').mean()

# # plot one year for sanity check
# ds_yearly.isel(time=0)['pm25_pred'].plot()

# save to geotiff

for time in ds_yearly.time.values:

    # select pm25 variable and year for file
    data_out = ds_yearly.sel(time=time)['pm25_pred']

    # full filename
    filename_out = os.path.join(out_data_path, 'financial_yearly', f'PM25_{ pd.to_datetime(time).strftime('%Y')}.tif')

    # save the file
    os.makedirs(os.path.dirname(filename_out), exist_ok=True)
    data_out.rio.to_raster(filename_out, driver='GTiff')

/home/work/.cache/pypoetry/virtualenvs/india-map-JHchGCJw-py3.12/lib/python3.12/site-packages/xarray/groupers.py:490: FutureWarning: 'AS-APR' is deprecated and will be removed in a future version, please use 'YS-APR' instead.
  self.index_grouper = pd.Grouper(


#### Monthly means

In [31]:
# calculate yearly mean
ds_monly = ds.resample(time='MS').mean()

# # plot one year for sanity check
# ds_monly.isel(time=0)['pm25_pred'].plot()

# save to geotiff

for time in ds_monly.time.values:

    # select pm25 variable and year for file
    data_out = ds_monly.sel(time=time)['pm25_pred']

    # full filename
    filename_out = os.path.join(out_data_path, 'monthly', f'PM25_{ pd.to_datetime(time).strftime('%Y%m')}.tif')

    # save the file
    os.makedirs(os.path.dirname(filename_out), exist_ok=True)
    data_out.rio.to_raster(filename_out, driver='GTiff')